<a href="https://colab.research.google.com/github/Elwing-Chou/ml0223/blob/main/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from zipfile import ZipFile
f = ZipFile("/content/drive/MyDrive/news/chinese_news_trans.zip")
f.extractall()
f = ZipFile("/content/drive/MyDrive/news/chinese_news_test.zip")
f.extractall()